In [ ]:
#conda install -c conda-forge pandas-profiling
#profile = ProfileReport(df, title="Pandas Profiling Report")
#conda install pandas-profiling==2.8.0

In [1]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

df = pd.read_csv(r'C:\Users\conno\Documents\School\Brazil\MAI_5003\Projetos\Home_Credit\Python\EDA\modeling_table_1.csv')

C:\Users\conno\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (87,90) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
df = df.drop(df.columns[0], axis=1)

In [3]:
# Split the datasets into Default & Non-Default
#df['TARGET'].describe()
#df['TARGET'].value_counts()

default_bool = df['TARGET'] == 1
clean_bool = df['TARGET'] == 0

default = df[default_bool]
clean = df[clean_bool]

In [10]:
#def_prof = ProfileReport(default)
#def_prof.to_file(output_file='Default_Profiling.html')
#def_prof = ProfileReport(default, title="Default_Profiling")
def_prof = default.profile_report(title='Default_Profiling',minimal=True)
def_prof.to_file(output_file='Default_Profiling.html')

In [11]:
#clean_prof = ProfileReport(clean)
#clean_prof.to_file(output_file='Non_Default_Profiling.html')
clean_prof = ProfileReport(clean, title="Non_Default_Profiling",minimal=True)
#clean_prof = clean.profile_report(title='Non_Default_Profiling')
clean_prof.to_file(output_file='Non_Default_Profiling.html')

In [17]:
def iv_woe(data, target, bins=10, show_woe=False):
    
    #Empty Dataframe
    newDF,woeDF = pd.DataFrame(), pd.DataFrame()
    
    #Extract Column Names
    cols = data.columns
    
    #Run WOE and IV on all the independent variables
    for ivars in cols[~cols.isin([target])]:
        if (data[ivars].dtype.kind in 'bifc') and (len(np.unique(data[ivars]))>10):
            binned_x = pd.qcut(data[ivars], bins,  duplicates='drop')
            d0 = pd.DataFrame({'x': binned_x, 'y': data[target]})
        else:
            d0 = pd.DataFrame({'x': data[ivars], 'y': data[target]})
        d = d0.groupby("x", as_index=False).agg({"y": ["count", "sum"]})
        d.columns = ['Cutoff', 'N', 'Events']
        d['% of Events'] = np.maximum(d['Events'], 0.5) / d['Events'].sum()
        d['Non-Events'] = d['N'] - d['Events']
        d['% of Non-Events'] = np.maximum(d['Non-Events'], 0.5) / d['Non-Events'].sum()
        d['WoE'] = np.log(d['% of Events']/d['% of Non-Events'])
        d['IV'] = d['WoE'] * (d['% of Events'] - d['% of Non-Events'])
        d.insert(loc=0, column='Variable', value=ivars)
        #print("Information value of " + ivars + " is " + str(round(d['IV'].sum(),6)))
        temp =pd.DataFrame({"Variable" : [ivars], "IV" : [d['IV'].sum()]}, columns = ["Variable", "IV"])
        newDF=pd.concat([newDF,temp], axis=0)
        woeDF=pd.concat([woeDF,d], axis=0)

        #Show WOE Table
        if show_woe == True:
            print(d)
    return newDF, woeDF

In [26]:
iv, woe = iv_woe(data=df,target='TARGET',bins=10,show_woe=False)

iv = iv.sort_values(by='IV',ascending=False)
iv.to_csv('Information_Value.csv')